In [1]:
import pandas as pd
import time
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import os

In [2]:
df = pd.read_csv('RC_books.csv' , encoding='latin-1')

C:\Users\student\AppData\Local\Temp\ipykernel_9480\946941554.py:1: DtypeWarning: Columns (12,13,14,15,16,17,18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('RC_books.csv' , encoding='latin-1')


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36358 entries, 0 to 36357
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Acc. Date           36179 non-null  object 
 1   Acc. No.            36358 non-null  int64  
 2   Title               36358 non-null  object 
 3   ISBN                36358 non-null  object 
 4   Author/Editor       36295 non-null  object 
 5   Ed./Vol.            5779 non-null   object 
 6   Place & Publisher   36358 non-null  object 
 7   Year                36188 non-null  float64
 8   Page(s)             36357 non-null  object 
 9   Class No./Book No.  36358 non-null  object 
 10  Unnamed: 10         23 non-null     object 
 11  Unnamed: 11         20 non-null     object 
 12  Unnamed: 12         13 non-null     object 
 13  Unnamed: 13         13 non-null     object 
 14  Unnamed: 14         11 non-null     object 
 15  Unnamed: 15         10 non-null     object 
 16  Unna

## Drop Unnamed Column

In [4]:
df_unnamed = df[['Unnamed: 10', 'Unnamed: 11','Unnamed: 12' , 'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16', 'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20']]

df = df.drop(columns= df_unnamed)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36358 entries, 0 to 36357
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Acc. Date           36179 non-null  object 
 1   Acc. No.            36358 non-null  int64  
 2   Title               36358 non-null  object 
 3   ISBN                36358 non-null  object 
 4   Author/Editor       36295 non-null  object 
 5   Ed./Vol.            5779 non-null   object 
 6   Place & Publisher   36358 non-null  object 
 7   Year                36188 non-null  float64
 8   Page(s)             36357 non-null  object 
 9   Class No./Book No.  36358 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 2.8+ MB


In [7]:
INPUT_CSV = "RC_books.csv"
OUTPUT_CSV = "books_with_description_using_googlebooksapi.csv"
ISBN_COL = "ISBN"

SLEEP_TIME = 0.2
MAX_RETRIES = 3
SAVE_EVERY = 200

# LOAD DATA
df = pd.read_csv(INPUT_CSV, encoding="latin1", low_memory=False)

if "book_description" not in df.columns:
    df["book_description"] = None
    df["description_source"] = None

# GOOGLE BOOKS FETCH
def fetch_google_books(isbn):
    if pd.isna(isbn):
        return None

    url = f"https://www.googleapis.com/books/v1/volumes?q=isbn:{isbn}"

    for _ in range(MAX_RETRIES):
        try:
            r = requests.get(url, timeout=5)
            if r.status_code == 200:
                data = r.json()
                if "items" in data:
                    return data["items"][0]["volumeInfo"].get("description")
        except:
            pass

        time.sleep(SLEEP_TIME)

    return None

for idx, isbn in enumerate(df[ISBN_COL]):

    if pd.notna(df.at[idx, "book_description"]):
        continue

    desc = fetch_google_books(isbn)
    if desc:
        df.at[idx, "book_description"] = desc
        df.at[idx, "description_source"] = "google_books"

    time.sleep(SLEEP_TIME)

    if (idx + 1) % SAVE_EVERY == 0:
        df.to_csv(OUTPUT_CSV, index=False)
        print(f"Saved at row {idx + 1}")

df.to_csv(OUTPUT_CSV, index=False)
print("Google Books fetching completed")


Saved at row 200
Saved at row 400
Saved at row 600
Saved at row 800
Saved at row 1000
Saved at row 1200
Saved at row 1400
Saved at row 1600
Saved at row 1800
Saved at row 2000
Saved at row 2200
Saved at row 2400
Saved at row 2600
Saved at row 2800
Saved at row 3000
Saved at row 3200
Saved at row 3400
Saved at row 3600
Saved at row 3800
Saved at row 4000
Saved at row 4200
Saved at row 4400
Saved at row 4600
Saved at row 4800
Saved at row 5000
Saved at row 5200
Saved at row 5400
Saved at row 5600
Saved at row 5800
Saved at row 6000
Saved at row 6200
Saved at row 6400
Saved at row 6600
Saved at row 6800
Saved at row 7000
Saved at row 7200
Saved at row 7400
Saved at row 7600
Saved at row 7800
Saved at row 8000
Saved at row 8200
Saved at row 8400
Saved at row 8600
Saved at row 8800
Saved at row 9000
Saved at row 9200
Saved at row 9400
Saved at row 9600
Saved at row 9800
Saved at row 10000
Saved at row 10200
Saved at row 10400
Saved at row 10600
Saved at row 10800
Saved at row 11000
Saved at